In [1]:
%config IPCompleter.greedy=True
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence,text
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle


Using TensorFlow backend.
C:\Users\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

In [3]:
# Data yükleme, boş satıları silme, aynı satıları temizleme, silinen satırlardan dolayı bozulan indexsi yeniden yaratma
filename="hepsi"
data = pd.read_excel('./'+filename+'.xlsx')
data.dropna( inplace = True)
duplicateDFRow = data[data.duplicated(['tweet'])]
data.drop_duplicates(subset ="tweet",keep = False, inplace = True)
data = data.reset_index(drop=True)
tags = data["tags"]
texts = data["tweet"]
data = shuffle(data)
data.head()

,tweet,tags
50,arıyoruz insana ulaşamıyoruz komple ailecek vo...,negatif
871,yaptığınız kampanyadan internetin hızı düştü d...,negatif
981,turkcell hakkını yedirmem,pozitif
465,vodafone operatörün internet çekmez ararım doğ...,negatif
788,türkiyenin büyük soygun gasp dolandırıcılık çe...,negatif


In [4]:
len(data)

1671

In [5]:
h=data['tags']=='negatif'
print("Negatif count" ,len(data[h]))
h=data['tags']=='pozitif'
print("Pozitif count" ,len(data[h]))

Negatif count 1420
Pozitif count 251


In [6]:
### Tagsleri 0 ve 1 yapma
le = LabelEncoder()
tags = le.fit_transform(tags)


In [7]:
# Kelime numalarandırmada en büyük sayı
num_max = 10000
## The process of enumerating words
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)


In [8]:
# En uzun tweet ve farklı kelime sayısı
max_len = 15
vocab_size=10000


In [9]:
# For example, how to enumerate words
print(texts[10])

for item in texts[10].split():    
    print(tok.word_index[item])



karantina bitince ilk isim vodafone aboneliğinden çıkmak açılmıyor vodafone yanımda uygulamasına giremiyorum yardım edin
171
790
42
353
1
1167
1168
486
1
32
293
294
200
201


In [10]:
## Tüm tweetleri numaralandırma
cnn_texts_seq = tok.texts_to_sequences(texts)


In [11]:
# Numaralandırılmış tweetleri belirli bir uzunluğa getirme snra padding
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len,padding='post')

# Örnek
print('***************************************************')
print(texts[50])
print(cnn_texts_mat[50])
print('***************************************************')


***************************************************
arıyoruz insana ulaşamıyoruz komple ailecek vodafone çıkalım istiyorsunuz numaranızı farklı operatöre taşıdık fatura geliyor konu hakkında yardımcı olun yasal yollara başvuracağım bilginize
[ 817 1206  363   71 2120   31  206  500  614   62  155 2121 2122  818
 2123]
***************************************************


In [12]:
# Hiç eğitime dahil etmeden test için veri ayırma
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(cnn_texts_mat,tags,test_size=0.1)


In [13]:
df = pd.DataFrame (X_train)
df.to_excel(filename+'-X_train.xlsx', index=False)

df = pd.DataFrame (X_test)
df.to_excel(filename+'-X_test.xlsx', index=False)
           
df = pd.DataFrame (y_train)
df.to_excel(filename+'-y_train.xlsx', index=False)

df = pd.DataFrame (y_test)
df.to_excel(filename+'-y_test.xlsx', index=False)

In [14]:
len(df)

168

In [22]:
df = pd.DataFrame (X_train,y_train)